In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [2]:
print('Num GPUs Avail: ',len(tf.config.list_physical_devices('GPU')))
os.getcwd()
#https://www.tensorflow.org/text/tutorials/text_generation
#https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb

Num GPUs Avail:  1


'C:\\Users\\vrchee\\Documents\\MSDS\\NLP\\chee-v-hw-4'

In [3]:
#path_to_file='emma_book_only.txt'
path_to_file='sanditon.txt'

In [4]:
text=open(path_to_file,'rb').read().decode(encoding='utf-8')

print(f'Length of text: {len(text)} characters') 

Length of text: 129249 characters


In [5]:
print(text[:250])

Chapter 1
A gentleman and a lady travelling from Tunbridge towards that part of the Sussex coast which lies between Hastings and Eastbourne, being induced by business to quit the high road and attempt a very rough lane, were overturned in toiling up


In [6]:
vocab=sorted(set(text))

print(f'{len(vocab)} unique characters')

79 unique characters


In [7]:
'''Process Data'''

chars=tf.strings.unicode_split(text, input_encoding='UTF-8')
chars

<tf.Tensor: shape=(129249,), dtype=string, numpy=array([b'C', b'h', b'a', ..., b'a', b'm', b'.'], dtype=object)>

In [8]:
ids_from_chars=preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

ids=ids_from_chars(chars)

In [9]:
chars_from_ids=tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars=chars_from_ids(ids)

In [10]:
tf.strings.reduce_join(chars, axis=-1).numpy()

def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [11]:
'''Prediction Task'''
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))
    

seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))


for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text


split_input_target(list("Tensorflow"))

dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

C
h
a
p
t
e
r
 
1

tf.Tensor(
[b'C' b'h' b'a' b'p' b't' b'e' b'r' b' ' b'1' b'\r' b'\n' b'A' b' ' b'g'
 b'e' b'n' b't' b'l' b'e' b'm' b'a' b'n' b' ' b'a' b'n' b'd' b' ' b'a'
 b' ' b'l' b'a' b'd' b'y' b' ' b't' b'r' b'a' b'v' b'e' b'l' b'l' b'i'
 b'n' b'g' b' ' b'f' b'r' b'o' b'm' b' ' b'T' b'u' b'n' b'b' b'r' b'i'
 b'd' b'g' b'e' b' ' b't' b'o' b'w' b'a' b'r' b'd' b's' b' ' b't' b'h'
 b'a' b't' b' ' b'p' b'a' b'r' b't' b' ' b'o' b'f' b' ' b't' b'h' b'e'
 b' ' b'S' b'u' b's' b's' b'e' b'x' b' ' b'c' b'o' b'a' b's' b't' b' '
 b'w' b'h' b'i'], shape=(101,), dtype=string)
b'Chapter 1\r\nA gentleman and a lady travelling from Tunbridge towards that part of the Sussex coast whi'
b'ch lies between Hastings and Eastbourne, being induced by business to quit the high road and attempt '
b'a very rough lane, were overturned in toiling up its long ascent, half rock, half sand. The accident '
b"happened just beyond the only gentleman's house near the lane\xe2\x80\x94a house which their driver, on be

In [12]:
'''Create Training batches'''
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [13]:
'''Build the Model'''
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [14]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [15]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [16]:
'''Try Model'''
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
    
model.summary()

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

(64, 100, 80) # (batch_size, sequence_length, vocab_size)
Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  20480     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  82000     
Total params: 4,040,784
Trainable params: 4,040,784
Non-trainable params: 0
_________________________________________________________________
Input:
 b'is! It was impossible not to feel him hardly used: to be obliged to stand back in his own house and '

Next Char Predictions:
 b'zu,jHPr22ymgpovHx").\xc3\xaaPGTjEY!0n?-hvi- ih\nkkUP?J4\xc3\xaa\xc3\xa0R0DH\rjJy45\'UcFtPh BoA"R\xc3\xa8l;\xc3\xa0yN\'GWk5DEEv:NqnxDq3yJbtn'


In [17]:
'''Train the Model'''
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 80)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.382318


In [18]:
tf.exp(mean_loss).numpy()
model.compile(optimizer='adam', loss=loss)

In [19]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [20]:
EPOCHS=50

In [21]:
history=model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
19/19 [==============================] - 2s 26ms/step - loss: 3.9563
Epoch 2/50
19/19 [==============================] - 1s 26ms/step - loss: 2.9924: 0s - loss: 3.0
Epoch 3/50
19/19 [==============================] - 1s 26ms/step - loss: 2.7088
Epoch 4/50
19/19 [==============================] - 1s 26ms/step - loss: 2.4793
Epoch 5/50
19/19 [==============================] - 1s 26ms/step - loss: 2.3582
Epoch 6/50
19/19 [==============================] - 1s 26ms/step - loss: 2.2824
Epoch 7/50
19/19 [==============================] - 1s 26ms/step - loss: 2.2253
Epoch 8/50
19/19 [==============================] - 1s 26ms/step - loss: 2.1673
Epoch 9/50
19/19 [==============================] - 1s 26ms/step - loss: 2.1107
Epoch 10/50
19/19 [==============================] - 1s 26ms/step - loss: 2.0495
Epoch 11/50
19/19 [==============================] - ETA: 0s - loss: 1.988 - 1s 26ms/step - loss: 1.9879
Epoch 12/50
19/19 [==============================] - 1s 25ms/step - loss: 1.92

In [22]:
'''GENERATE TEXT'''
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()
    
        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask
    
        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
    
        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)
    
        # Return the characters and model state.
        return predicted_chars, states

In [23]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [25]:
start = time.time()
states = None
next_char = tf.constant(['It was impossible not to feel him hardly used: to be obliged to stand back in his own house and see the best place by the fire constantly occupied by Sir Harry Denham.'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

It was impossible not to feel him hardly used: to be obliged to stand back in his own house and see the best place by the fire constantly occupied by Sir Harry Denham."

"And his house wese to make as tall a place in tawing-to her; and my dear Tom, upon I have no fancy for herself. Not as well as I have. And I do believe those are best on fide with her relation for tuins, the poor to fancy themselves or spirits to speak in a common. I should like to have you two,

Mrs. Parker, who, in her first really volusemes of probably steps at Sanditon, it would be better if they would leave themselves undeasonably indisposed to Vielly ojje's account. have such fortitude!" and Charlotte asseding anything more than anything. I have viewede and just as well be done."

"Oh, me a was evidently a sentance like Sanditon. The sea air would probubly be the death of me. And neither do every portrywloms! Hervey were extreme young man, however, and that but not to add the empassood of Sid Edward's right to i

In [ ]:
'''Export the Generator'''
#tf.saved_model.save(one_step_model, 'one_step')
#one_step_reloaded = tf.saved_model.load('one_step')
#
#states = None
#next_char = tf.constant(['ROMEO:'])
#result = [next_char]
#
#for n in range(100):
#    next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
#    result.append(next_char)
#
#print(tf.strings.join(result)[0].numpy().decode("utf-8"))